# Import Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

In [2]:
# Reading the data
data = pd.read_excel('../data/responses_data.xlsx')

In [3]:
# Checking the head of the data
data.head()

,question_id,description,num_response,label_in_question,answer_id,answer_category_num,answer_justification,answer_upvote,account_id_id,question_id_id,date
0,4,ما فيك تعيش بلا ما تكب فتن ليل نهار وبكرة قلهم...,0,normal,147,Normal,none were given,0,52,4,2022-12-03
1,37,ههههههه داعشي طنط كول هوا ولاك حريمه,0,hate,326,Mockery,none were given,0,45,37,2022-12-19
2,47,هؤلاء الخبثاء والخنازير أينما حلو يحل بارضهم ا...,0,hate,76,Violent,none were given,0,8,47,2022-11-19
3,2,سورية بلد الحضارات تربطها بعلية او بحيوان,2,normal,11,Racist,none were given,0,1,2,2022-09-09
4,2,سورية بلد الحضارات تربطها بعلية او بحيوان,2,normal,17,Normal,none were given,0,7,2,2022-09-19


In [4]:
data.shape

(500, 11)

In [5]:
data.description = data.description.apply(str.strip)

In [6]:
# The shape of the data
data.shape

(500, 11)

In [7]:
# The columns of the data
data.columns

Index(['question_id', 'description', 'num_response', 'label_in_question',
       'answer_id', 'answer_category_num', 'answer_justification',
       'answer_upvote', 'account_id_id', 'question_id_id', 'date'],
      dtype='object')

In [8]:
# checking how many comments are repeated
i = 0
for comment in data.description.unique():
    temp_df = data[data.description == comment]
    if temp_df.shape[0] >= 2:
        
        i+=1
print(i)

26


In [9]:
clean_data = data.copy()
clean_data  = clean_data[['description', 'answer_category_num']]
clean_data.head(2)

,description,answer_category_num
0,ما فيك تعيش بلا ما تكب فتن ليل نهار وبكرة قلهم...,Normal
1,ههههههه داعشي طنط كول هوا ولاك حريمه,Mockery


In [10]:
clean_data.description.value_counts()

وزير الخارجية اللبناني جبران باسيل قال في سلسلة تغريداته عقب اختتام القمة العربية التنموية الاقتصادية والاجتماعية لمسنا تج…    7
سورية بلد الحضارات تربطها بعلية او بحيوان                                                                                      6
لما العاهرة تاضر بالعفة                                                                                                        4
انت بدك طبيب نفسي                                                                                                              2
احلى تحية للجزائر وشعبها                                                                                                       2
                                                                                                                              ..
موفقين ونحن معكم وسنقرا لكم                                                                                                    1
إستحوا بقى                                                                                       

In [11]:
# removing duplicated descrptions/ comments
clean_data.drop_duplicates(subset='description', inplace=True)
clean_data.reset_index(drop=True, inplace=True)

In [12]:
clean_data

,description,answer_category_num
0,ما فيك تعيش بلا ما تكب فتن ليل نهار وبكرة قلهم...,Normal
1,ههههههه داعشي طنط كول هوا ولاك حريمه,Mockery
2,هؤلاء الخبثاء والخنازير أينما حلو يحل بارضهم ا...,Violent
3,سورية بلد الحضارات تربطها بعلية او بحيوان,Racist
4,سكربينة عنصرية حقها فرنك فوتي حسابها و شوفي؛بس...,Mockery
...,...,...
458,هلا غزة الوحيدة بكل فلسطين هي الحرة... لهيك بص...,Violent
459,آآآخ على القهر ودنائة حكام العرب واقفلهم كلاب ...,Violent
460,عندما يأتي اليوم يحترم اللبنانيين بعضهم البعضو...,Normal
461,نحتاج الى مسؤولين جدد يعملو على اعادة الوطن ال...,Normal


In [13]:
# checking how many comments have different votes
for i, row in clean_data.iterrows():

    comment = row.description

    # getting the dataframe for that comment
    temp_df = data[data.description == comment]
    
    # how many unique answers that comment has
    n_ = temp_df.answer_category_num.nunique()

    # changing the answer_category_num to the most frequent one
    if n_ >= 2:
        most_voted_label = temp_df.answer_category_num.value_counts().index[0]
        clean_data.at[i, 'answer_category_num'] = most_voted_label

In [14]:
clean_data

,description,answer_category_num
0,ما فيك تعيش بلا ما تكب فتن ليل نهار وبكرة قلهم...,Normal
1,ههههههه داعشي طنط كول هوا ولاك حريمه,Mockery
2,هؤلاء الخبثاء والخنازير أينما حلو يحل بارضهم ا...,Violent
3,سورية بلد الحضارات تربطها بعلية او بحيوان,Religious affiliation
4,سكربينة عنصرية حقها فرنك فوتي حسابها و شوفي؛بس...,Mockery
...,...,...
458,هلا غزة الوحيدة بكل فلسطين هي الحرة... لهيك بص...,Violent
459,آآآخ على القهر ودنائة حكام العرب واقفلهم كلاب ...,Violent
460,عندما يأتي اليوم يحترم اللبنانيين بعضهم البعضو...,Normal
461,نحتاج الى مسؤولين جدد يعملو على اعادة الوطن ال...,Normal


In [15]:
# Generated Class
clean_data.answer_category_num.value_counts()

Normal                   211
Mockery                  124
Violent                   71
Racist                    34
Religious affiliation     14
Sexual harrasment          9
Name: answer_category_num, dtype: int64

In [16]:
# Checking the info
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   description          463 non-null    object
 1   answer_category_num  463 non-null    object
dtypes: object(2)
memory usage: 7.4+ KB


We can see that there is no missing values

In [17]:
# Removing stop words for each tweet
clean_data['tweet_no_stopwords'] = 'x'
stop_words = set(stopwords.words('arabic')) 
for count, tweet in enumerate(clean_data.description):
    word_tokens = word_tokenize(tweet)
    filtered_tweet = []
    for word in word_tokens:
        if word not in stop_words:
            filtered_tweet.append(word)
    joined_filtered_tweet = " ".join(filtered_tweet)
    clean_data.tweet_no_stopwords[count] = joined_filtered_tweet

In [18]:
clean_data

,description,answer_category_num,tweet_no_stopwords
0,ما فيك تعيش بلا ما تكب فتن ليل نهار وبكرة قلهم...,Normal,فيك تعيش بلا تكب فتن ليل نهار وبكرة قلهم الموض...
1,ههههههه داعشي طنط كول هوا ولاك حريمه,Mockery,ههههههه داعشي طنط كول هوا ولاك حريمه
2,هؤلاء الخبثاء والخنازير أينما حلو يحل بارضهم ا...,Violent,الخبثاء والخنازير حلو يحل بارضهم الخراب
3,سورية بلد الحضارات تربطها بعلية او بحيوان,Religious affiliation,سورية بلد الحضارات تربطها بعلية او بحيوان
4,سكربينة عنصرية حقها فرنك فوتي حسابها و شوفي؛بس...,Mockery,سكربينة عنصرية حقها فرنك فوتي حسابها شوفي؛بس خ...
...,...,...,...
458,هلا غزة الوحيدة بكل فلسطين هي الحرة... لهيك بص...,Violent,غزة الوحيدة بكل فلسطين الحرة ... لهيك بصبو حقد...
459,آآآخ على القهر ودنائة حكام العرب واقفلهم كلاب ...,Violent,آآآخ القهر ودنائة حكام العرب واقفلهم كلاب مسلح...
460,عندما يأتي اليوم يحترم اللبنانيين بعضهم البعضو...,Normal,عندما يأتي اليوم يحترم اللبنانيين بعضهم البعضو...
461,نحتاج الى مسؤولين جدد يعملو على اعادة الوطن ال...,Normal,نحتاج الى مسؤولين جدد يعملو اعادة الوطن الى ال...


# Training the Model

In [19]:
# y is Class which is dependent on X Tweet
X = clean_data['tweet_no_stopwords']
y = clean_data['answer_category_num']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [20]:
y_train.value_counts()

Normal                   177
Mockery                  101
Violent                   66
Racist                    29
Religious affiliation     11
Sexual harrasment          9
Name: answer_category_num, dtype: int64

In [21]:
y_test.value_counts()

Normal                   34
Mockery                  23
Racist                    5
Violent                   5
Religious affiliation     3
Name: answer_category_num, dtype: int64

In [22]:
# Using tf-idf 
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train)
X_train_tfidf.shape

(393, 2094)

In [23]:
# Using pipiles for machine learning flow

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),])


text_clf.fit(X_train, y_train)  


predictions = text_clf.predict(X_test)

In [24]:
# Confusion Matrix
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[ 5 11  2  0  5]
 [ 2 28  0  1  3]
 [ 0  2  0  0  3]
 [ 0  2  0  1  0]
 [ 1  2  0  0  2]]


In [25]:
# Classification report
print(metrics.classification_report(y_test,predictions)) 

                       precision    recall  f1-score   support

              Mockery       0.62      0.22      0.32        23
               Normal       0.62      0.82      0.71        34
               Racist       0.00      0.00      0.00         5
Religious affiliation       0.50      0.33      0.40         3
              Violent       0.15      0.40      0.22         5

             accuracy                           0.51        70
            macro avg       0.38      0.35      0.33        70
         weighted avg       0.54      0.51      0.48        70



In [26]:
# Accuracy
print(metrics.accuracy_score(y_test,predictions))

0.5142857142857142


In [25]:
tst =  "انت  واحد حقير وما بتستحي ا"

In [26]:
text_clf.predict([tst])[0]

'Mockery'

In [27]:
for count, test_tweet in enumerate(X_test):
    print(count)
    print("Tweet: {}".format(test_tweet))
    print("Real Prediction: {} ||  Model Prediction: {}".format(y_test.values[count], predictions[count]))

0
Tweet: اشم ريحت زباله يمكن سني حقير
Real Prediction: Racist ||  Model Prediction: Violent
1
Tweet: هاي ميّل غزّيل غزّيل ميّل
Real Prediction: Normal ||  Model Prediction: Mockery
2
Tweet: عوى شي جديد
Real Prediction: Mockery ||  Model Prediction: Normal
3
Tweet: كأنك ابنه عم دافع عنه بيخلق
Real Prediction: Normal ||  Model Prediction: Normal
4
Tweet: كنت سوري يمكن كنا عذرناك انك تكون اعرابي وجاي تاكل هوا هون بحط البوط بتمك ولاك
Real Prediction: Racist ||  Model Prediction: Violent
5
Tweet: عنجد بهيم هلق قطر أودعت المبلغ شان الفوائد
Real Prediction: Mockery ||  Model Prediction: Racist
6
Tweet: ردود الشعب اللبناني كفاية رئيس الحزب توحيد العرب قال انت معروف مذهبك مره ومره ضد ولك
Real Prediction: Normal ||  Model Prediction: Normal
7
Tweet: روح تعالج حمار انت
Real Prediction: Mockery ||  Model Prediction: Mockery
8
Tweet: التعليقات اهتمت بالعنوان وتجاوزت المضمون فقال وهذا صحيح فنحن الجزائر ايام ازمتنا الامنية اصطف الفلسطينيون العرب ضدنا وقاطعونا فعرفات الذى انلم يتغدى الجزائر يتعشى قاطع

# Saving the Model

In [27]:
import pickle
# save the model to disk
filename = 'finalized_model_SVC.sav'
pickle.dump(text_clf, open(filename, 'wb'))

In [74]:
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.8393162393162393
